# PyTorch Land Use Try

In [1]:
import numpy as np
from torch.utils import data
import torch
from __future__ import print_function
import argparse
import torch
import numpy as np
from torch.nn import functional
import torch.optim as optim
from torch.utils import data
from unet import UNet
from torchvision import transforms
from torch.autograd import Variable
from skimage import exposure
from skimage.io import imread
from skimage.util import *
import matplotlib.pyplot as plt
import natsort as ns
import os
from tqdm import tqdm
from collections import OrderedDict

In [2]:
from unet import UNet

In [3]:
class ZurichLoader(data.Dataset):
    """
    Data loader for Zurich dataset
    """
    def __init__(self, im_patches, gt_patches, split, data_augmentation = False):
        """
        Load data.
        :param split: 'train', 'val' or empty
        :param transform_data: list of data transformations for images
        :param transform_labels: list of data transformations for labels
        :param im_size: size to which to crop labels and images
        :param patch_size: size of image patches
        """
        # data transformations
        self.data_augmentation = data_augmentation

        # Load image indexes, depending on set:
        if split == 'train':
            self.img_idx = np.arange(1, int(len(im_patches)*.6))
        elif split == 'val':
            self.img_idx = np.arange(int(len(im_patches)*.6)+1, int(len(im_patches)*.8))
        else:
            self.img_idx = np.arange(int(len(im_patches)*.8)+1, int(len(im_patches)))

        self.im_patches = [im_patches[i] for i in self.img_idx]
        self.gt_patches= [gt_patches[i] for i in self.img_idx]

        # translate to data and label paths

    def __getitem__(self, idx):
        """
        function must be overridden: returns data-label pair of tensors for data point at index
        Here we just return the entire images for demonstration reasons. In reality, you would crop
        from each image at random here, or would have a pre-defined list of coordinates initialised
        in the constructor and crop according to it.
        """

        img = self.im_patches[idx]
        gt = self.gt_patches[idx]

        # convert image
        #img = Image.fromarray((img*255).astype(np.uint8))
        #gt = Image.fromarray(gt.astype(np.uint8)).convert('L')
        # apply transformations


        if self.data_augmentation:
            img, gt = augment_images_and_gt(img, gt)


        # If you want to do special transforms like rotation, do them here.
        # Don't forget to apply the same transforms to both the data and label tensors.
        # You can use Torchsample, or else convert the data to numpy (e.g.: img.numpy())
        # and then load it again into a torch tensor (img = torch.from_numpy(img)).

        # TODO transformations using torchsample
        img = np.asarray(img).transpose((2,0,1)).astype(np.float64)
        img = torch.from_numpy(img).type(torch.FloatTensor)
        gt[gt==0] = -1
        gt = torch.from_numpy(gt).type(torch.LongTensor)  # .astype(np.double))

        return img, gt

    def __len__(self):
        # function must be overridden: returns number of data points in data set
        return len(self.gt_patches)

In [ ]:

def get_gt_patches(images_gt, patch_size=64):
    """
    get ground truth patches for all images
    """
    patches = []
    for im in tqdm(images_gt):
        patches_im_gt = view_as_blocks(im, block_shape=(patch_size, patch_size))
        n_patches = int((im.shape[0] / patch_size) ** 2)  # 25*25 = 625 per image
        patches_im_gt = np.reshape(patches_im_gt, (n_patches, patch_size, patch_size))

        patches.append(patches_im_gt)
    patches = np.array(patches)
    patches = np.asarray([patches[i][j] for i in range(len(patches)) for j in range(len(patches[i]))])
    # patches = np.concatenate(patches, axis = 0)
    return np.asarray(patches)


def get_padded_patches(images, patch_size=16, window_size=64):
    """
    get padded (mirror) patches for all images
    """
    patches = []
    for im in tqdm(images):
        patches_im = np.zeros(
            [int(im.shape[0] / patch_size), int(im.shape[0] / patch_size), window_size, window_size, im.shape[-1]])
        for i in range(im.shape[-1]):
            padded = np.lib.pad(im[:, :, i], int(np.floor((window_size - patch_size) / 2)), 'reflect')
            patches_im[:, :, :, :, i] = view_as_windows(padded, window_size, step=patch_size)

        n_patches = int((im.shape[0] / patch_size) ** 2)  # 25*25 = 625 per image
        patches_im = np.reshape(patches_im, (n_patches, window_size, window_size, im.shape[2]))

        patches.append(patches_im)
    patches = np.array(patches)
    patches = np.asarray([patches[i][j] for i in range(len(patches)) for j in range(len(patches[i]))])
    # patches = np.concatenate(patches, axis = 0)
    return patches


# Training settings
batch_size = 2
test_batch_size = 2
epochs = 5
lr = 0.001
momentum = 0.5
no_cuda = True
seed = 1
log_interval = 20

cuda = not no_cuda and torch.cuda.is_available()

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}


# define transforms


def im_load(path, max_size=512):  # for now, only return highest [max_size] pixels, multiple of patch_size
    """load a TIF image"""
    image = np.asarray(imread(path)).astype('double')
    image = image[:max_size, :max_size, :]
    return image


def imgs_stretch_eq(im):
    """
    Do contrast stretching on image
    :param im: input image
    :return img_stretch: stretched image
    :return img_eq: equalized image
    """
    im = np.asarray(im)
    img_stretch = im.copy()
    img_eq = im.copy()
    for band in range(im.shape[-1]):
        p2, p98 = np.percentile(im[:, :, band], (2, 98))
        img_stretch[:, :, band] = exposure.rescale_intensity(im[:, :, band], in_range=(p2, p98))
        img_eq[:, :, band] = exposure.equalize_hist(img_stretch[:, :, band])
    return img_stretch, img_eq


legend = OrderedDict((('Background', [255, 255, 255]),
                      ('Roads', [0, 0, 0]),
                      ('Buildings', [100, 100, 100]),
                      ('Trees', [0, 125, 0]),
                      ('Grass', [0, 255, 0]),
                      ('Bare Soil', [150, 80, 0]),
                      ('Water', [0, 0, 150]),
                      ('Railways', [255, 255, 0]),
                      ('Swimming Pools', [150, 150, 255])))

# get class names by increasing value (as done above)
names, colors = [], []
for name, color in legend.items():
    names.append(name)
    colors.append(color)


def gt_color_to_label(gt, maj=False):
    """
    Transform a set of GT image in value range [0, 255] of shape (n_images, width, height, 3)
    to a set of GT labels of shape (n_images, width, height)
    """

    # sum of distinct color values
    gt_new = np.zeros(np.asarray(gt).shape[:-1])

    # replace colors by new values
    for i in range(len(colors)):
        gt_new[np.all(gt == colors[i], axis=-1)] = i  # np.argsort(colors)[i]

    if maj:
        # return only majority label for each patch
        gt_maj_label = []
        for i in range(len(gt)):
            counts = np.bincount(gt_new[i].flatten())
            gt_maj_label.append(np.argmax(counts))

        gt_new = np.asarray([gt_maj_label]).T

    return gt_new


def gt_label_to_color(gt):
    """
    Transform a set of GT labels of shape (n_images, width, height)
    to a set of GT images in value range [0,1] of shape (n_images, width, height, 3) """
    gt_new = np.zeros(gt.shape + (3,))
    for i in range(len(colors)):  # loop colors
        gt_new[gt == i, :] = np.divide(colors[i], 255)
    return gt_new


def load_data(max_size=512, patch_size=64):
    """
    load image data and create patches to use with DataLoader
    :return:
    """
    base_dir = "/home/cyrilwendl/SIE-Master/Zurich/Zurich_dataset"
    im_dir = base_dir + '/images_tif/'
    gt_dir = base_dir + '/groundtruth/'
    im_names = ns.natsorted(os.listdir(im_dir))
    gt_names = ns.natsorted(os.listdir(gt_dir))
    print("images: %i " % len(im_names))
    print("ground truth images: %i " % len(gt_names))

    imgs = np.asarray([im_load(im_dir + im_name, max_size=max_size) for im_name in im_names])
    gt = np.asarray([im_load(gt_dir + gt_name, max_size=max_size) for gt_name in gt_names])

    # image stretching
    imgs =[imgs_stretch_eq(img)[1] for img in imgs]



    # convert gt colors to labels
    gt_maj_label = gt_color_to_label(gt)
    gt = gt_maj_label

    # get patches
    im_patches = get_padded_patches(imgs, patch_size=patch_size, window_size=patch_size)
    gt_patches = get_gt_patches(gt, patch_size=patch_size)

    return im_patches, gt_patches


im_patches, gt_patches = load_data(max_size=512, patch_size=64)

transform_data = transforms.Compose([
    transforms.ToTensor()])

transform_both = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()])

# create datasets
train_loader = data.DataLoader(
    ZurichLoader(im_patches, gt_patches, 'train'),
    batch_size=batch_size, shuffle=True)

val_loader = data.DataLoader(
    ZurichLoader(im_patches, gt_patches, 'val'),
    batch_size=batch_size, shuffle=True)

test_loader = data.DataLoader(
    ZurichLoader(im_patches, gt_patches, 'test'),
    batch_size=test_batch_size, shuffle=True)

n_classes = 8  # TODO parse

model = UNet(n_classes, in_channels=4)

if cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)


def train(epochs=epochs):
    model.train()
    for batch_idx, (im_data, labels) in enumerate(train_loader):
        # print("Shape batch:"+str(np.shape(Variable(im_data).data.numpy())))
        # print("Batch id: %i"%batch_idx)
        if cuda:
            im_data, labels = im_data.cuda(), labels.cuda()
        im_data, labels = Variable(im_data), Variable(labels)
        # class_weights = class_weight.compute_class_weight('balanced', np.unique(labels.data.numpy().flatten()),
        #                                                np.arange(10))
        # class_weights=Variable(torch.from_numpy(class_weights).type(torch.FloatTensor))
        optimizer.zero_grad()
        output = model(im_data)
        loss = functional.nll_loss(output, labels, ignore_index=-1)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epochs, batch_idx * len(im_data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for im_data, target in test_loader:
        if cuda:
            im_data, target = im_data.cuda(), target.cuda()
        im_data, target = Variable(im_data), Variable(target)
        output = model(im_data)
        test_loss += functional.nll_loss(output, target, size_average=False, ignore_index=-1).data[0]  # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset) * 64 * 64 * 4,
        100. * correct / (len(test_loader.dataset) * 64 * 64 * 4)))


def test_show_some_images():
    test_im = test_loader.dataset[11]
    im_test = Variable(test_im[0]).data.numpy()
    im_test_l = Variable(test_im[1]).data.numpy()

    print(im_test.shape, im_test_l.shape)

    im_test = im_test * 1.
    im_test = np.transpose(im_test, (1, 2, 0))
    im_test_l = im_test_l * 1.
    #print(np.shape(im_test_l))

    plt.figure()
    plt.imshow(im_test[:, :, :3])
    plt.show()


    plt.figure()
    plt.imshow(im_test_l)
    plt.show()


if __name__ == '__main__':

    for epoch in range(1, epochs + 1):
        train(epoch)
        test()
        #test_show_some_images()
        #break

images: 20 
ground truth images: 20 


100%|██████████| 20/20 [00:00<00:00, 1185.70it/s]


Train Epoch: 1 [0/767 (0%)]	Loss: -0.026590
Train Epoch: 1 [40/767 (5%)]	Loss: -0.041622
Train Epoch: 1 [80/767 (10%)]	Loss: -0.084711
Train Epoch: 1 [120/767 (16%)]	Loss: -0.116668
Train Epoch: 1 [160/767 (21%)]	Loss: -0.210624
